<a href="https://colab.research.google.com/github/khalidelboray/ML_Course/blob/master/last_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests pandas==1.3.3 numpy==1.21.2 matplotlib==3.4.3 plotly==5.3.1 scikit_learn==0.24.2 seaborn==0.11.2

     |████████████████████████████████| 11.3 MB 7.5 MB/s 
     |████████████████████████████████| 15.7 MB 193 kB/s 
     |████████████████████████████████| 10.3 MB 21.9 MB/s 
     |████████████████████████████████| 23.9 MB 11 kB/s 
     |████████████████████████████████| 22.3 MB 1.5 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
  Attempting uninstall: plotly
    Found e

In [2]:
import matplotlib as plt
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [6]:
df = pd.read_csv("https://raw.githubusercontent.com/khalidelboray/ML_Course/master/data/new_dataset.csv")
df.head()

,Unnamed: 0,gender,social_group,parent_education,lunch,test_prep_course,math_score,read_score,write_score,average,grade
0,0,0,1,1,1,1,72,72,74,72.666667,1
1,1,0,2,4,1,0,69,90,88,82.333333,0
2,2,0,1,3,1,1,90,95,93,92.666667,0
3,3,1,0,0,0,1,47,57,44,49.333333,4
4,4,1,2,4,1,1,76,78,75,76.333333,1
